In [1]:
# Order preparation for NSE

# STATUS: Completed
# Run-time: 10 seconds

# Dependencies:
# /zdata/*.pkl - for pickles generated by 01_nse_scan program

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 3000, clientId=11) # kavi tws live
# ib = IB().connect('127.0.0.1', 3000, clientId=11) # kavi IBG live

In [2]:
%%time
import pandas as pd
import numpy as np
import datetime
from os import listdir

#... set limits and penalties
    
m_maxp = 0.015    # % of max margin allowed on net liquidity per scrip to limit positon risk
    
min_rom = 0.25     # 0.5 would be 50% return

base = 0.05       # Upper or Lower base multiple for prices

max_nlvp = 0.8    # max allowable nlv to prevent overall portfolio risk. 0.8 means 80% of NLV.
                  # max available funds for option trades = max_nlvp * NLV - initMargin
    
max_dte = 65     # Maximum horizon for option expiry

sigma = 0.2      # from the mean

#... read the account info
ac = ib.accountSummary()
df_a = util.df(ac)

#... set max margin per position
net_liq = float(df_a[df_a.tag == 'NetLiquidation'].iloc[0].value) 
av_funds = float(df_a[df_a.tag == 'FullAvailableFunds'].iloc[0].value)
max_p = net_liq*m_maxp

#****    PREPARE TO HARVEST   ****
#_________________________________

#... read the positions
ps = ib.portfolio()
df_p = util.df(ps)

df_p['ibSymbol'] = [s.symbol for s in df_p.contract.values]

# get expected price percentage from DTE
def expPricePct(expiry):
    '''Gets expected price percentage from DTE for harvesting trades.
    Assumes max DTE to be 30 days.
    Arg: (expiry) as string 'yyymmdd', e.g. from expPricePct 
    Returns: expected price percentage (xpp) as float
    Ref: http://interactiveds.com.au/software/Linest-poly.xls ... for getting curve function
    '''
#     if dte is to be extracted from contract.lastTradeDateOrContractMonth
    dte = (util.parseIBDatetime(expiry) - datetime.datetime.now().date()).days
    
    if dte > 30:
        dte = 30  # Forces the max DTE to be 30 days
    
    xpp = (103.6008 - 3.63457*dte + 0.03454677*dte*dte)/100
    
    return xpp

# get the harvest as lower of discount from curve * averageCost and discount * marketPrice

expiry = [d.lastTradeDateOrContractMonth for d in df_p.contract]
discount = [m for m in map(expPricePct, expiry)]
df_p['hvstPrice'] = pd.concat([df_p.averageCost*discount, 
                               df_p.marketPrice*(1-np.array(discount))], axis=1).min(axis=1)

df_p.hvstPrice = np.floor(df_p.hvstPrice/base)*base # round down to the nearest 0.05

df_p.loc[df_p.hvstPrice == 0, 'hvstPrice'] = base  # make the 0s to 5 paise

# harvest open positions with hvstPrice
qual_contracts = ib.qualifyContracts(*df_p.contract)
df_p['qual_contracts'] = qual_contracts
df_p['harvestOrder'] = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) for position, hvstPrice in zip(df_p.position, df_p.hvstPrice)]

#****   PREPARE TO SOW !    ****
#________________________________

#... get the lots and margins
# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MM', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

df_slm = pd.merge(df_p, df_paisa[['ibSymbol', 'Mlot', 'TotMgnPerShr']])

#... make the blacklist

df1 = df_slm.groupby('ibSymbol').sum()
df1['used_margin'] = -df1.position * df1.TotMgnPerShr
df1['max_margin'] = max_p
df1['avail_margin'] = df1.position * df1.TotMgnPerShr + df1.max_margin

df1['max_units'] = (df1.avail_margin/(df1.Mlot*df1.TotMgnPerShr)).apply(np.floor)

blacklist = list(df1[df1.max_units <= 0].index)

#... build the high-pop-roc dataframe

# read the dataframe pickles from zdata. Ignore the underscores (underlying)
fs = listdir('./zdata/')
fs = [f for f in fs if (f[-3:] == 'pkl') & (f[0] != '_')]

dfs = pd.concat([pd.read_pickle(r'./zdata/'+f) for f in fs], axis=0, sort=True).reset_index(drop=True).sort_values('rom', ascending=False)

# filter out puts which are not in blacklist
dfs_p = dfs[(dfs.right=='P') & (~dfs.symbol.isin(blacklist))]

# filter out puts whose underlying price is greater than minimum rom
df2 = dfs_p[(dfs_p.rom > min_rom)]

# filter puts with dte < 65 days
df3 = df2[df2.dte <= max_dte]

# filter puts whose underlying price is above the mean.
df4 = df3[(df3.undPrice > df3.avg) & (df3.strike < (df3.avg - df3.stDev*sigma))]

# filter puts with strikes below lo
# df5 = df4[df4.strike < df4.lo]

Wall time: 7.31 s


In [ ]:
# len(df4)
df4[['symbol', 'strike', 'dte', 'right', 'undPrice', 'pop', 'rom', 'expPrice', 'hv', 'bsmDelta', 'bsmPrice', 'ibiv', 'ibdelta', 'ibprice', 'close', 'bid', 'ask', 'lo', 'avg', 'stDev']]

In [ ]:
# WARNING!!!... Make df the dataframe that you want to execute on!
df = df4.copy()   # make this the last dataframe to get the orders placed

contracts = [c.contract for c in df.opt_ticker]

df.loc[df.expPrice < 0.2, 'expPrice'] = 0.2  # Make the selling price a minimum of 0.2
orders = [LimitOrder(action='SELL', totalQuantity=lot, lmtPrice=expPrice) for lot, expPrice in zip(df.lot, df.expPrice)]

print('{:d} contracts from {:d} scrips, consuming {:,} margin from full available funds of {:,}'.format(len(contracts), len(df.symbol.unique()), sum(df.margin), av_funds*max_nlvp))

In [ ]:
df_analyze = df[df.symbol.isin(df.symbol.unique())].groupby(['dte', 'symbol', 'strike']).apply(max)

df_analyze.to_excel('./zdata/analyze.xlsx', index=None, header=True)

# Write to watchlist. This watchlist is to be checked in tws / tradingview for the lowest strike and expiry
# This needs to be imported to IBKR's watchlist
watchlist = [('DES', s, 'STK', 'NSE') for s in df.symbol.unique()]
df_watch = util.df(watchlist)

df_watch.to_csv('./zdata/watchlist.csv', index=None, header=False)

In [ ]:
# Write closed orders placed to file
x  =  r'./zdata/' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_closing_orders.xlsx"
writer = pd.ExcelWriter(x)
df_p.to_excel(writer, 'all-options', index=False, header=1)
writer.save()

In [3]:
df_a

,account,tag,value,currency,modelCode
0,U9329809,AccountType,INDIVIDUAL,,
1,U9329809,Cushion,0.59841,,
2,U9329809,LookAheadNextChange,1548150300,,
3,U9329809,AccruedCash,0.00,INR,
4,U9329809,AvailableFunds,11802222.25,INR,
5,U9329809,BuyingPower,23604444.49,INR,
6,U9329809,EquityWithLoanValue,23698632.36,INR,
7,U9329809,ExcessLiquidity,14181504.27,INR,
8,U9329809,FullAvailableFunds,11802222.25,INR,
9,U9329809,FullExcessLiquidity,14181504.27,INR,


In [22]:
ib.accountSummary()

[AccountValue(account='U9329809', tag='AccountType', value='INDIVIDUAL', currency='', modelCode=''),
 AccountValue(account='U9329809', tag='Cushion', value='0.598424', currency='', modelCode=''),
 AccountValue(account='U9329809', tag='LookAheadNextChange', value='1548150300', currency='', modelCode=''),
 AccountValue(account='U9329809', tag='AccruedCash', value='0.00', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='AvailableFunds', value='11801639.37', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='BuyingPower', value='23603278.74', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='EquityWithLoanValue', value='23696640.91', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='ExcessLiquidity', value='14180639.68', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='FullAvailableFunds', value='11801639.37', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='FullExcessL

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivi

In [21]:
ib.fills()

[Fill(contract=Option(conId=339368706, symbol='NHPC', lastTradeDateOrContractMonth='20190131', strike=14.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='NHPC19JAN14PE', tradingClass='NHPC'), execution=Execution(execId='00016710.5c46802d.01.01', time=datetime.datetime(2019, 1, 22, 3, 45, 7, tzinfo=datetime.timezone.utc), acctNumber='U9329809', exchange='NSE', side='BOT', shares=27000.0, price=0.05, permId=1539942780, clientId=11, orderId=795, cumQty=27000.0, avgPrice=0.05, lastLiquidity=1), commissionReport=CommissionReport(execId='00016710.5c46802d.01.01', commission=24.4235, currency='INR', realizedPNL=3995.9825), time=datetime.datetime(2019, 1, 22, 3, 45, 7, tzinfo=datetime.timezone.utc)),
 Fill(contract=Option(conId=343966255, symbol='ADANIPORT', lastTradeDateOrContractMonth='20190228', strike=355.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ADANIPORTS19FEB355PE', tradingClass='ADANIPORTS'), execution=Execution(execId='0001671

In [20]:
sum([(f.execution.price * f.execution.shares) for f in ib.fills() if f.execution.side == 'SLD'])

24775.0

In [ ]:
[f.contract.symbol, f.]